# Introduction to labeled dimensions: why do we need them?

In [ ]:
%matplotlib widget
import numpy as np
import scipp as sc
from utils.helper import plot, fill_up_container

rng = np.random.default_rng(seed=1234)

In [ ]:
ny, nx = 10, 20
x = np.linspace(0, 2 * np.pi, nx)
y = np.linspace(0, 2 * np.pi, ny)
x, y = np.meshgrid(x, y)

z = np.sin(x) * np.cos(y)

In [ ]:
# What do we know about the variable `z`?
z.shape

In [ ]:
plot(z)

In [ ]:
# Slice out row number 4
plot(z[4, :])

In [ ]:
# Slice out col number 12
plot(z[12, :])

In [ ]:
plot(z[:, 12])

#### We can't always deduce from the shape

When both dimensions have the same length,
it can sometimes be difficult to remember which dimension is which:

In [ ]:
ny, nx = 20, 20
x = np.linspace(0, 2 * np.pi, nx)
y = np.linspace(0, 2 * np.pi, ny)
x, y = np.meshgrid(x, y)

z = np.sin(x) * np.cos(y)

In [ ]:
plot(z)

In [ ]:
plot(z[4, :], z[:, 4])

### The situation gets worse with more dimensions

Say I now have an array that has 4 dimensions: `x, y, z, time` (in that order if I'm lucky?).

In [ ]:
a = np.random.random([20] * 4)
a.shape

I want to get the first `z` slice...

Which one was it again?

In [ ]:
z_slice = a[:, :, 0, :]  # x,y,z,t
z_slice = a[0, :, :, :]  # z,y,x,t
z_slice = a[:, :, :, 0]  # t,x,y,z

# Introducing labeled dimensions

<img src="https://xarray.dev/Xarray-assets/RGB/Xarray_Logo_RGB_Final.svg" width="220" />

[Xarray](https://docs.xarray.dev/en/stable/index.html) (https://docs.xarray.dev) introduced labels to multi-dimensional Numpy arrays.

"*real-world datasets are usually more than just raw numbers; they have labels which encode information about how the array values map to locations in space, time, etc.*"

We have embraced, and to a large extent copied, the Xarray mechanism.

<img src="https://scipp.github.io/_static/logo-2022.svg" width="220" />

In [ ]:
var = sc.array(dims=["x", "y", "z", "time"], values=a)

In [ ]:
# Get the first slice in the dimension z
var["z", 0]

In [ ]:
# Get the first 10 slices in the dimension time
var["time", 0:10]

#### Adding coordinates

Coordinates can be specified for each dimension.
Essentially, they describe the extent of each axis, as well as how far each data point is from its neighbours.

In [ ]:
data = sc.array(dims=["latitude", "longitude"], values=rng.random((5, 9)))
sc.show(data)

In [ ]:
da = sc.DataArray(
    data=data,
    coords={
        "longitude": sc.linspace("longitude", -120, 120, 9),
        "latitude": sc.linspace("latitude", -70, 70, 5),
    },
)
da

In [ ]:
sc.show(da)

In [ ]:
da.plot()

### Automatic broadcasting

Because of the labeled dimensions,
operations between arrays with different dimensions can automatically broadcast operands to the correct shape:

In [ ]:
a = sc.array(dims=["y", "x"], values=rng.random((50, 50)))  # 2D array
b = sc.array(dims=["x"], values=np.arange(50.0))  # 1D array
c = a * b
c

In [ ]:
c.plot(aspect="equal")

There is no longer a need for Numpy's `x.reshape(50, 1)`!

In [ ]:
a = sc.array(dims=["y", "x"], values=rng.random((50, 50)))  # 2D array
b = sc.array(dims=["z"], values=np.arange(50.0))  # 1D array
c = a * b
c

# Physical units

Every data variable and coordinate in Scipp has physical units.

In [ ]:
s = rng.normal(size=(2, 10000))
h = np.histogram2d(s[0], s[1], bins=(50, 50))[0]

image = sc.array(dims=["y", "x"], values=h, unit="counts")
image

In [ ]:
image.plot(aspect="equal")

In [ ]:
integration_time = sc.scalar(300.0, unit="s")
image /= integration_time
print(image.unit)

image.plot(aspect="equal")

### Units also provide protection

Say I now have a background image (dark frame) which I want to subtract from the signal image above,
but I forgot to first normalize it by integration time

In [ ]:
background = sc.array(dims=["y", "x"], values=rng.random((50, 50)), unit="counts")

image - background

In [ ]:
background_integration_time = sc.scalar(60.0, unit="s")
background /= background_integration_time

(image - background).plot(aspect="equal")

In [ ]:
image.dims

In [ ]:
print(f"unit: {image.unit}")
print(f"dims: {image.dims}")
image

The units are very useful in early prevention of difficult-to-spot bugs in a workflow.

They save **hours** of debugging time, free-up mental capacity and let the user focus on the important thing: **doing science**.

(see also [pint](https://pint.readthedocs.io/en/stable/), [astropy.units](https://docs.astropy.org/en/stable/units/index.html), [pint-xarray](https://pint-xarray.readthedocs.io/en/stable/), ...)

In [ ]:
scalar = sc.scalar(1.2, unit="Å")
sc.show(scalar)
scalar

In [ ]:
sc.scalar(1.0, unit="g") + sc.scalar(1.0, unit="kg").to(unit="g")

:::{important} Let's make chia oat pudding with scipp

Make a dictionary of `ingredients` (scalars) with:
- [ ] 80g Oats
- [ ] 20g Chia seeds
- [ ] 160ml milk (I like oat milk)
- [ ] 5g date syrup
- [ ] 1g salt
:::

:::{note} Solution
:class: dropdown
```
ingredients = {
    'oats': sc.scalar(80, unit='g'),
    'chia': sc.scalar(20, unit='g'),
    'milk': sc.scalar(160, unit='ml'),
    'date': sc.scalar(5, unit='g'),
    'salt': sc.scalar(1, unit='g'),
}
```
:::

In [ ]:
ingredients = {
    'oats': ...,
    'chia': ...,
    'milk': ...,
    'date': ...,
    'salt': ...,
}

:::{admonition} Do we have everything?
{eval}`', '.join(f"{i} ({ingredients[i].value}{ingredients[i].unit})" for i in ingredients)`
:::

In [ ]:
# Let's model our pudding.
number_of_seeds = sc.scalar(600, unit="1/g")

In [ ]:
seeds = number_of_seeds * ingredients["chia"]
seeds

In [ ]:
container = sc.zeros(dims=("x", "y", "z"), shape=(39, 39, 39))

In [ ]:
container

In [ ]:
fill_up_container(container, seeds)

In [ ]:
container.sum()

:::{important} Let's look at our pudding.

Make plots using the dimensions (`x`, `y`, `z`) which
- is the 10th slice in the `x` dimension.
- plots the 15th slice in `y` dimension and between the 0 and 15 indices in `z` axis.
- finds all the seeds in the `z` dimension for `x=18, y=20`.
:::

:::{note} Solution
:class: dropdown
```
container['x', 10].plot()
container['y', 15]['z', 0:15].plot()
container['x', 18]['y', 20].plot()
```

:::

:::{admonition} But what about the full container?
:class: tip

Let's plot it!

:::


In [ ]:
import plopp as pp

In [ ]:
data = sc.DataArray(
    data=container,
    coords={
        "x": sc.linspace("x", 0, 39, 39),
        "y": sc.linspace("y", 0, 39, 39),
        "z": sc.linspace("z", 0, 39, 39),
    },
)

In [ ]:
pp.scatter3d(data, cmap="binary")